In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
from dipy.reconst import mapmri
from dipy.viz import fvtk
from dipy.data import get_sphere
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np
import json
from diGP.preprocessing import get_HCP_loader

Read the data for this tutorial.

MAPMRI requires multi-shell data, to properly fit the radial part of the basis.

In [6]:
with open('../config.json', 'r') as json_file:
    conf = json.load(json_file)

subject_path = conf['HCP']['data_paths']['mgh_1007']

In [8]:
loader = get_HCP_loader(subject_path)
small_data_path = '{}/mri/small_data.npy'.format(subject_path)

loader.update_filename_data(small_data_path)

data = loader.data
gtab = loader.gtab
voxel_size = loader.voxel_size

(10, 10, 10, 552)

Instanstiate the MAPMRI Model.

In [10]:
radial_order = 6
map_model_laplacian_aniso = mapmri.MapmriModel(gtab, radial_order=radial_order,
                                               laplacian_regularization=True,
                                               laplacian_weighting=.2)

We can then fit the MAPMRI model to the data.

In [12]:
mapfit_laplacian_aniso = map_model_laplacian_aniso.fit(data)

### Compute the ODFs

Load an odf reconstruction sphere

In [13]:
sphere = get_sphere('symmetric724')

The radial order s can be increased to sharpen the results, but it might
also make the odfs noisier. Always check the results visually.

In [14]:
odf = mapfit_laplacian_aniso.odf(sphere, s=2)
print('odf.shape (%d, %d, %d, %d)' % odf.shape)

odf.shape (10, 10, 10, 724)


## Display the ODFs

In [32]:
r = fvtk.ren()
sfu = fvtk.sphere_funcs(odf[:,:, 5:6, :], sphere, colormap='jet')
#sfu.RotateX(-30)
fvtk.add(r, sfu)
fvtk.record(r, n_frames=1, out_path='odfs.png', size=(600, 600))